In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import pickle

df = pd.read_csv('MagicBricks_Updated_Clear_Locality_draft15122023.csv')

In [2]:
Q1 = df['Price'].quantile(0.10)
Q3 = df['Price'].quantile(0.90)

PQ1 = df['Per_Sqft'].quantile(0.10)
PQ3 = df['Per_Sqft'].quantile(0.90)

AQ1 = df['Area'].quantile(0.10)
AQ3 = df['Area'].quantile(0.90)

In [3]:
df1=df[(df['Price']>Q1) & (df['Price']<Q3) & (df['Per_Sqft']>PQ1) & (df['Per_Sqft']<PQ3) & (df['Area']>AQ1) & (df['Area']<AQ3)]

In [4]:
df1=df1.dropna()

In [5]:
threshold_low = 6000000
threshold_high = 10000000

# Function to compare values with thresholds
def compare_with_threshold(value):
    if value < threshold_low:
        return 'Low'
    elif threshold_low <= value <= threshold_high:
        return 'Medium'
    else:
        return 'High'

# Apply the function to create a new column
df1['Price_range'] = df1['Price'].apply(compare_with_threshold)

# Display the DataFrame
print(df1)

        Area  BHK  Bathroom      Furnishing                   Locality  \
1      750.0    2       2.0  Semi-Furnished           Rohini Sector 24   
2      950.0    2       2.0       Furnished           Rohini Sector 13   
3      600.0    2       2.0  Semi-Furnished           Rohini Sector 24   
4      650.0    2       2.0  Semi-Furnished           Rohini Sector 24   
5     1300.0    4       3.0  Semi-Furnished           Rohini Sector 24   
...      ...  ...       ...             ...                        ...   
1252  1800.0    3       3.0  Semi-Furnished                    Kalkaji   
1253  1200.0    3       3.0  Semi-Furnished          Chittaranjan Park   
1255  1050.0    3       2.0  Semi-Furnished          Chittaranjan Park   
1256   875.0    3       3.0  Semi-Furnished          Chittaranjan Park   
1257   990.0    2       2.0     Unfurnished  Chittaranjan Park Block A   

      Parking     Price         Status   Transaction           Type  Per_Sqft  \
1         1.0   5000000  Ready

In [6]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

columns_to_encode = ['Furnishing', 'Status', 'Transaction', 'Type','Locality','Price_range']

for column in columns_to_encode:
    df1[column] = label_encoder.fit_transform(df1[column])

df1.head()

,Area,BHK,Bathroom,Furnishing,Locality,Parking,Price,Status,Transaction,Type,Per_Sqft,Price_range
1,750.0,2,2.0,1,55,1.0,5000000,1,0,0,6667.0,1
2,950.0,2,2.0,0,47,1.0,15500000,1,1,0,6667.0,0
3,600.0,2,2.0,1,55,1.0,4200000,1,1,1,6667.0,1
4,650.0,2,2.0,1,55,1.0,6200000,1,0,1,6667.0,2
5,1300.0,4,3.0,1,55,1.0,15500000,1,0,1,6667.0,0


In [7]:
X=df1.drop(["Price_range",'Price'],axis=1)
y=df1['Price_range']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=33)

classifiers = {
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC(),
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Naive Bayes': GaussianNB()
}

for name, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name}: Accuracy - {accuracy:.4f}")

Random Forest: Accuracy - 0.8322
Support Vector Machine: Accuracy - 0.7063
Logistic Regression: Accuracy - 0.7273
K-Nearest Neighbors: Accuracy - 0.7622
Decision Tree: Accuracy - 0.7972
Naive Bayes: Accuracy - 0.6853


C:\Users\prasa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [10]:
pickle.dump(model,open('model_house_pricing','wb'))